## La longitudine determina un maggior numero di goal segnati nei campionati tedeschi?

Esercizio

- Si considerano due campionati nazionali tedeschi: [Bundesliga](https://it.wikipedia.org/wiki/Bundesliga) e [2. Fußball-Bundesliga](https://it.wikipedia.org/wiki/2._Fu%C3%9Fball-Bundesliga) nelle stagioni che vanno dalla 1999-2000 alla 2018-2019.

- Per ciascuna squadra partecipante a ciascuna delle edizioni, ricavare qual è la città della squadra e la sua longitudine (in maniera automatica, procedurale).

- Secondo quanto visto nel notebook 'regressione.ipynb', provare un modello di regressione lineare che spieghi il numero di goal fatti da una squadra in funzione della sua longitudine, calcolando anche R2. 
- Svolere un approfondimento sulla divisione fra campione di train e campione di validazione (es. la seconda serie mi spiega la prima serie? il primo decennio mi spiega il secondo decennio?)
- Variando il modo con cui si prendono gli insiemi di train e di validazione, si possono spiegare alcune evoluzioni del calcio tedesco e anche un eventuale miglioramento del calcio nella Germania Est col passare del tempo.
- Provare poi il modello di Poisson.

- Commentare i risultati



In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

classifica = []
teams = {}
cities = {}

numero_colonne = 11
numero_squadre = 18

In [11]:
def citta_squadra(nomeSquadra):
    citta = teams[nomeSquadra]
    
    return "squadra:" + nomeSquadra + " --> città:" + citta + " --> latitudine:" + cities[citta]['latitudine'] + \
           " longitudine:" + cities[citta]['longitudine']


def lettura_link_stagioni(campionato,selettore,linkalbo,inizio,fine):
    page = requests.get(linkalbo)
    soup = BeautifulSoup(page.content, 'html.parser')
    page.close()
    albo = soup.select(selettore)

    for i in range(len(albo)):
        info = str(albo[i]).split(">")
        paginaWeb = "https://it.wikipedia.org/" + info[0].split('"')[1];
        data = info[1].split("<")[0].split("-")
        if len(data[0])  == 2 : data[0] = '20' + data[0]
        if len(data[1])  == 2 : data[1] = '20' + data[1]
        start = int(data[0])
        end = int(data[1])
        
        if start >= inizio and end <= fine :
            campionati.append([data[0] + "-" + data[1], campionato, paginaWeb])

def info_squadra_citta(squadra,sitoSquadra):
   
    page = requests.get(sitoSquadra)

    row = str(BeautifulSoup(page.content, 'html.parser').find(href='/wiki/Citt%C3%A0').parent.parent.select('td:last-child')).split("href")
    info_city = row[len(row) - 1].split(">")
    linkCitta = info_city[0].split('"')[1]
    citta =info_city[1].split('<')[0]
    page.close()
    teams[squadra] = citta

    if citta not in cities.keys():
        sitoCitta = 'https://it.wikipedia.org' + linkCitta
        pageCitta = requests.get(sitoCitta)
        latitudine = str(BeautifulSoup(pageCitta.content, 'html.parser').select("span.latitude")[0]).\
        split(">")[1].split("<")[0]

        longitudine = str(BeautifulSoup(pageCitta.content, 'html.parser').select("span.longitude")[0]).\
                      split(">")[1].split("<")[0]

        pageCitta.close()
        cities[citta] = {'latitudine':latitudine,'longitudine':longitudine}


def stagione(campionato):
    
    stop = 0
    paginaCampionato = requests.get(campionato[2])
    soupCampionato = BeautifulSoup(paginaCampionato.content, 'html.parser')
    Classifica = str(soupCampionato.find('span', {"id":['Classifica_finale','Classifica_Finale','Classifica']}).\
                     parent.find_next("table")).split("</tr>")[1:numero_squadre + 1]
    
    for rigaClassifica in Classifica:
        punteggi_squadra = [campionato[0],campionato[1]]
        
        riga = rigaClassifica.split("</td>")[1:]
        
        #posiizone
        pos = int(riga[0].split("<td>")[1].split(".")[0].split('*')[0].split("\n")[0])
        punteggi_squadra.append(pos)

        #squadra 
        info_squadra = riga[1].split('<a href=')[1].split(">") 
        squadra = info_squadra[1].split('<')[0]  
        punteggi_squadra.append(squadra)


        #punti finali
        punti = int(riga[2].split('<b>')[1].split('</b>')[0].split('*')[0].split("\n")[0])
        punteggi_squadra.append(punti)

        for i in range(3,numero_colonne - 2):
             punteggi_squadra.append(int(riga[i].split('<td>')[1].split('*')[0].split("\n")[0]))
            
        punteggi_squadra.append(int(riga[7].split('<td>')[1].split('*')[0].split("\n")[0]) - int(riga[8].split('<td>')[1].split('*')[0].split("\n")[0]))

        if squadra not in teams.keys():
            sito = 'https://it.wikipedia.org' + info_squadra[0].split('"')[1]
            info_squadra_citta(squadra,sito)

        classifica.append(punteggi_squadra)   
        print(punteggi_squadra)
        stop += 1
        if stop == numero_squadre  :break;
    paginaCampionato.close()    



def stagione_2007_2008_bundesliga_2(campionato):

    paginaCampionato = requests.get(campionato[2])
    soupCampionato = BeautifulSoup(paginaCampionato.content, 'html.parser')
    Classifica = str(soupCampionato.find('span', {"id": 'Classifica_finale'}).parent.find_next("table")).split("<tr")[2:numero_squadre + 2]

    for c in Classifica:
        riga = [campionato[0],campionato[1]]
        col = c.split("<td")
        pos = int(col[1].split('>')[1].split(".")[0])
        riga.append(pos)

        info_squadra = col[2].split("href=")[1].split(">")
        squadra = info_squadra[1].split("</a")[0]
        riga.append(squadra)

        punti = int(col[10].split("<b>")[1].split('</b>')[0])
        riga.append(punti)


        for i in range(3,9):
            riga.append(int(str(col[i].split(">")[1].split('\n')[0].split('</td')[0])))

        riga.append(int(str(col[7].split(">")[1].split('\n')[0].split('</td')[0])) - int(str(col[8].split(">")[1].split('\n')[0].split('</td')[0])))
        
        if squadra not in teams.keys():
            sito = 'https://it.wikipedia.org' + info_squadra[0].split('"')[1]
            info_squadra_citta(squadra,sito)
        
        print(riga)
        classifica.append(riga)


In [12]:
campionati = []
classifica = []
# lista pagine campionati dal 1999
lettura_link_stagioni('Bundesliga','div.responsive-columns>div:nth-child(3)>ul>li>a:first-child',"https://it.wikipedia.org/wiki/Bundesliga",1999,2019)

#bundesliga
for campionato in campionati:
    stagione(campionato)

#bundesliga 2
campionati = []
lettura_link_stagioni('Bundesliga 2','div.responsive-columns>div:nth-child(2)>ul>li>a:first-child',"https://it.wikipedia.org/wiki/2._Fu%C3%9Fball-Bundesliga",1999,2007)
for campionato in campionati:
    stagione(campionato)


campionato = ['2007-2008','Bundesliga 2','https://it.wikipedia.org/wiki/2._Fußball-Bundesliga_2007-2008']
stagione_2007_2008_bundesliga_2(campionato)

campionati = []
lettura_link_stagioni('Bundesliga 2','div.responsive-columns>div:nth-child(2)>ul>li>a:first-child',"https://it.wikipedia.org/wiki/2._Fu%C3%9Fball-Bundesliga",2008,2019)
for campionato in campionati:
    stagione(campionato)

['1999-2000', 'Bundesliga', 1, 'Bayern Monaco', 73, 34, 22, 7, 5, 73, 28, 45]
['1999-2000', 'Bundesliga', 2, 'Bayer Leverkusen', 73, 34, 21, 10, 3, 74, 36, 38]
['1999-2000', 'Bundesliga', 3, 'Amburgo', 59, 34, 16, 11, 7, 63, 39, 24]
['1999-2000', 'Bundesliga', 4, 'Monaco 1860', 53, 34, 14, 11, 9, 55, 48, 7]
['1999-2000', 'Bundesliga', 5, 'Kaiserslautern', 50, 34, 15, 5, 14, 54, 59, -5]
['1999-2000', 'Bundesliga', 6, 'Hertha Berlino', 50, 34, 13, 11, 10, 39, 46, -7]
['1999-2000', 'Bundesliga', 7, 'Wolfsburg', 49, 34, 12, 13, 9, 51, 58, -7]
['1999-2000', 'Bundesliga', 8, 'Stoccarda', 48, 34, 14, 6, 14, 44, 47, -3]
['1999-2000', 'Bundesliga', 9, 'Werder Brema', 47, 34, 13, 8, 13, 65, 52, 13]
['1999-2000', 'Bundesliga', 10, 'Unterhaching', 44, 34, 12, 8, 14, 40, 42, -2]
['1999-2000', 'Bundesliga', 11, 'Borussia Dortmund', 40, 34, 9, 13, 12, 41, 38, 3]
['1999-2000', 'Bundesliga', 12, 'Friburgo', 40, 34, 10, 10, 14, 45, 50, -5]
['1999-2000', 'Bundesliga', 13, 'Schalke 04', 39, 34, 8, 15, 11,

In [67]:
dati = []
for c in classifica:
    dati.append(c)

df_classifiche= pd.DataFrame(data=dati,columns=['Stagione', 'Campionato', 'Posizione','Squadra','Punti','Giocate','Vinte','Pareggiate',
                                                'Perse','Goal Fatti','Goal Subiti','Differenza Goal'])
df_classifiche

,Stagione,Campionato,Posizione,Squadra,Punti,Giocate,Vinte,Pareggiate,Perse,Goal Fatti,Goal Subiti,Differenza Goal
0,1999-2000,Bundesliga,1,Bayern Monaco,73,34,22,7,5,73,28,45
1,1999-2000,Bundesliga,2,Bayer Leverkusen,73,34,21,10,3,74,36,38
2,1999-2000,Bundesliga,3,Amburgo,59,34,16,11,7,63,39,24
3,1999-2000,Bundesliga,4,Monaco 1860,53,34,14,11,9,55,48,7
4,1999-2000,Bundesliga,5,Kaiserslautern,50,34,15,5,14,54,59,-5
...,...,...,...,...,...,...,...,...,...,...,...,...
715,2018-2019,Bundesliga 2,14,Erzgebirge Aue,40,34,11,7,16,43,47,-4
716,2018-2019,Bundesliga 2,15,Sandhausen,38,34,9,11,14,45,52,-7
717,2018-2019,Bundesliga 2,16,Ingolstadt 04,35,34,9,8,17,43,55,-12
718,2018-2019,Bundesliga 2,17,Magdeburgo,31,34,6,13,15,35,53,-18


In [17]:
df_classifiche.isna().any()


Stagione           False
Campionato         False
Posizione          False
Squadra            False
Punti              False
Giocate            False
Vinte              False
Pareggiate         False
Perse              False
Goal Fatti         False
Goal Subiti        False
Differenza Goal    False
dtype: bool

In [29]:
df_cities = pd.DataFrame.from_dict(cities, orient='index')
df_cities

,latitudine,longitudine
Monaco di Baviera,48°08′13.9″N,11°34′31.8″E
Leverkusen,51°02′N,6°59′E
Amburgo,53°33′02″N,9°59′36″E
Kaiserslautern,49°26′41″N,7°46′08″E
Berlino,52°31′N,13°23′E
...,...,...
Sandhausen,49°20′38″N,8°39′29″E
Heidenheim an der Brenz,48°40′34″N,10°09′16″E
Würzburg,49°47′40″N,9°55′46″E
Kiel,54°19′24″N,10°08′22″E


In [62]:
df_cities['long_norm']=df_cities.apply(lambda x: int(x['longitudine'].split('°')[0])+int(x['longitudine'].split('°')[1].split('′')[0])/60, axis=1)
df_cities.reset_index(inplace=True)
df_cities

,index,latitudine,longitudine,long_norm
0,Monaco di Baviera,48°08′13.9″N,11°34′31.8″E,11.566667
1,Leverkusen,51°02′N,6°59′E,6.983333
2,Amburgo,53°33′02″N,9°59′36″E,9.983333
3,Kaiserslautern,49°26′41″N,7°46′08″E,7.766667
4,Berlino,52°31′N,13°23′E,13.383333
...,...,...,...,...
60,Sandhausen,49°20′38″N,8°39′29″E,8.650000
61,Heidenheim an der Brenz,48°40′34″N,10°09′16″E,10.150000
62,Würzburg,49°47′40″N,9°55′46″E,9.916667
63,Kiel,54°19′24″N,10°08′22″E,10.133333


In [41]:
#df_cities.reset_index()
df_cities.isna().any()

latitudine     False
longitudine    False
long_norm      False
dtype: bool

In [42]:
df_cities

,latitudine,longitudine,long_norm
Monaco di Baviera,48°08′13.9″N,11°34′31.8″E,11.566667
Leverkusen,51°02′N,6°59′E,6.983333
Amburgo,53°33′02″N,9°59′36″E,9.983333
Kaiserslautern,49°26′41″N,7°46′08″E,7.766667
Berlino,52°31′N,13°23′E,13.383333
...,...,...,...
Sandhausen,49°20′38″N,8°39′29″E,8.650000
Heidenheim an der Brenz,48°40′34″N,10°09′16″E,10.150000
Würzburg,49°47′40″N,9°55′46″E,9.916667
Kiel,54°19′24″N,10°08′22″E,10.133333


In [60]:
df_teams = pd.DataFrame.from_dict(teams, orient='index',columns=['Città'])
df_teams

,Città
Bayern Monaco,Monaco di Baviera
Bayer Leverkusen,Leverkusen
Amburgo,Amburgo
Monaco 1860,Monaco di Baviera
Kaiserslautern,Kaiserslautern
...,...
Sandhausen,Sandhausen
Heidenheim,Heidenheim an der Brenz
Kickers Würzburg,Würzburg
Holstein Kiel,Kiel


In [61]:
df_teams.reset_index(inplace=True)
df_teams

,index,Città
0,Bayern Monaco,Monaco di Baviera
1,Bayer Leverkusen,Leverkusen
2,Amburgo,Amburgo
3,Monaco 1860,Monaco di Baviera
4,Kaiserslautern,Kaiserslautern
...,...,...
73,Sandhausen,Sandhausen
74,Heidenheim,Heidenheim an der Brenz
75,Kickers Würzburg,Würzburg
76,Holstein Kiel,Kiel


In [21]:
df_teams.isna().any()

Città    False
dtype: bool

In [22]:
citta_squadra('Bayern Monaco')

'squadra:Bayern Monaco --> città:Monaco di Baviera --> latitudine:48°08′13.9″N longitudine:11°34′31.8″E'

In [23]:
citta_squadra('Amburgo')

'squadra:Amburgo --> città:Amburgo --> latitudine:53°33′02″N longitudine:9°59′36″E'

In [63]:
df_teams.isna().any()

index    False
Città    False
dtype: bool

In [65]:
df_squadra_long=df_teams.merge(df_cities, left_on='Città', right_on='index')

In [66]:
df_squadra_long

,index_x,Città,index_y,latitudine,longitudine,long_norm
0,Bayern Monaco,Monaco di Baviera,Monaco di Baviera,48°08′13.9″N,11°34′31.8″E,11.566667
1,Monaco 1860,Monaco di Baviera,Monaco di Baviera,48°08′13.9″N,11°34′31.8″E,11.566667
2,Bayer Leverkusen,Leverkusen,Leverkusen,51°02′N,6°59′E,6.983333
3,Amburgo,Amburgo,Amburgo,53°33′02″N,9°59′36″E,9.983333
4,Kaiserslautern,Kaiserslautern,Kaiserslautern,49°26′41″N,7°46′08″E,7.766667
...,...,...,...,...,...,...
73,Sandhausen,Sandhausen,Sandhausen,49°20′38″N,8°39′29″E,8.650000
74,Heidenheim,Heidenheim an der Brenz,Heidenheim an der Brenz,48°40′34″N,10°09′16″E,10.150000
75,Kickers Würzburg,Würzburg,Würzburg,49°47′40″N,9°55′46″E,9.916667
76,Holstein Kiel,Kiel,Kiel,54°19′24″N,10°08′22″E,10.133333
